# Offset computation 
This notebook computes the offsets for the time synchronization for all the sensors in the array. The time synchronization procedure is as follows:
 - We insert a patter in the clock of the event camera, the VNAV, and the FLIR camera
 - We look for this pattern by looking at the timestamps in local clock
 - We compute the offset between the beginning of the pattern and the local clock
 - **We use the end of the pattern as the starting point** This is important so the number of messages is consistent between sensors.

In [ ]:
%load_ext autoreload
%autoreload 2
    
import sys
sys.path.append('..')
import ha_python_utils.constants as const
from  ha_python_utils.bag_reader import BagReader
from event_camera_py import Decoder as ECDecoder
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from event_camera_py import Decoder
import copy
import yaml
import datetime

In [ ]:
# Input bag
# Experiments on 08.19
# BAG = Path("/data/ha_ec_data/2024.08.19.Pennov.Flight.and.Calib/cam.calib.1/raw_bag/ha_ec_2024-08-19-12-52-56_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.19.Pennov.Flight.and.Calib/cam.calib.2/raw_bag/ha_ec_2024-08-19-12-55-58_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.19.Pennov.Flight.and.Calib/flight.day.1/raw_bag/ha_ec_2024-08-19-11-43-08_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.19.Pennov.Flight.and.Calib/flight.day.2/raw_bag/ha_ec_2024-08-19-12-11-57_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.19.Pennov.Flight.and.Calib/imu.calib.1/raw_bag/ha_ec_2024-08-19-13-00-10_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.19.Pennov.Flight.and.Calib/imu.calib.2/raw_bag/ha_ec_2024-08-19-13-02-06_0.mcap")
# Experiments on 08.25
# BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/imu.calib.1/raw_bag/ha_ec_2024-08-25-16-44-38_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/flight.day.4/raw_bag/ha_ec_2024-08-25-17-35-46_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/imu.calib.2/raw_bag/ha_ec_2024-08-25-16-46-30_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/flight.night.3/raw_bag/ha_ec_2024-08-25-20-12-45_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/flight.day.3/raw_bag/ha_ec_2024-08-25-17-15-12_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/cam.calib.1/raw_bag/ha_ec_2024-08-25-16-38-19_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/flight.day.1/raw_bag/ha_ec_2024-08-25-15-27-17_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/flight.night.1/raw_bag/ha_ec_2024-08-25-19-44-20_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/flight.day.2/raw_bag/ha_ec_2024-08-25-15-58-07_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/cam.calib.2/raw_bag/ha_ec_2024-08-25-16-41-21_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/imu.calib.3/raw_bag/ha_ec_2024-08-25-16-48-34_0.mcap")
# BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/flight.night.2/raw_bag/ha_ec_2024-08-25-19-58-42_0.mcap")


In [ ]:
# Global variables used all over the file
SYNC_YAML = BAG.parents[1] / "sync_info.yaml"

# Print statistics for the whole bag
bag_reader = BagReader(str(BAG), const.ALL_INPUT_TOPICS)
bag_reader.print_stats(all_topics=True)

## Computation of offset for FLIR camera

In [ ]:
# Create a bag reader only for FLIR msgs
bag_reader = BagReader(str(BAG), const.ALL_FLIR_TOPICS)
bag_reader.print_stats()

# Get number of channels for the FLIR camera
flir_number_raw = bag_reader.get_num_msgs(const.FLIR_TOPIC_RAW)
flir_number_info = bag_reader.get_num_msgs(const.FLIR_TOPIC_INFO)
flir_number_meta = bag_reader.get_num_msgs(const.FLIR_TOPIC_META)


# Number of messages for both channels should be the same
try:
    assert (flir_number_raw == flir_number_info == flir_number_meta)
except AssertionError:
    print(f"The number of messages does not match:")
    print(f"flir_raw: {flir_number_raw}")
    print(f"flir_info: {flir_number_info}")
    print(f"flir_meta: {flir_number_meta}")

num_msgs_flir = flir_number_meta
if (num_msgs_flir > const.TRIGGER_FREQ*60):
    num_msgs_flir = const.TRIGGER_FREQ*60
arr_camtime = np.zeros(num_msgs_flir, dtype=np.uint64)
arr_header_times = np.zeros((num_msgs_flir, 2), dtype=np.uint64)

# Create a bag reader only for meta msgs
bag_reader = BagReader(str(BAG), [const.FLIR_TOPIC_META])
for i, (topic, msg, t_rec)  in enumerate(bag_reader.read_all()):
    if (i == num_msgs_flir):
        break
    arr_camtime[i] = msg.camera_time - msg.exposure_time*1000
    # Log the header times to synchronize non-clocked sensors
    arr_header_times[i] = msg.header.stamp.sec, msg.header.stamp.nanosec
    
    if (i % 500 == 0):
        print(f"{i} - {i/num_msgs_flir}")

arr_camtime_double = arr_camtime.astype(np.double)/1e9 # In s

In [ ]:
# Calculate the sequence times for the camera
diff_ts = np.diff(arr_camtime_double-arr_camtime_double[0])
p99 = np.percentile(diff_ts, 99)
print(1/const.TRIGGER_FREQ, p99)
assert np.isclose(1/const.TRIGGER_FREQ, p99, atol=1e-6, rtol=1e-4)

# Calculate the indices of the peaks
peak_idx = np.where(diff_ts > 2*1/const.TRIGGER_FREQ)[0]
assert len(peak_idx) == 4

# The peak corresponds to the last sample before and after the silence
time_diffs_flir = np.diff(arr_camtime_double[peak_idx])

diff_with_golden_flir = time_diffs_flir - const.GOLDEN_DIFFS
print(f"Golden diffs: {const.GOLDEN_DIFFS}")
print(f"This bag diffs: {time_diffs_flir}")
print(f"Diff with golden in us: {diff_with_golden_flir*1e6}")

# Timing is good when is less than 10 us
assert(np.all(diff_with_golden_flir*1e6<10))

# Time offset is the time of the first diff
first_sample_flir = peak_idx[3]+1
#print(arr_camtime[first_sample_flir] - arr_camtime[first_sample_flir -1])
time_offset_flir = arr_camtime[first_sample_flir]
print(f"Time offset FLIR: {time_offset_flir}")
time_offset_ros_stamp = arr_header_times[first_sample_flir]

# Print timestamp of the first sample to double check
print(f"Timestamp of first sample: {arr_camtime[first_sample_flir] - time_offset_flir}") 
offset_wrt_start_trigger_flir = time_offset_flir -arr_camtime[peak_idx[0]]
assert np.isclose(const.GOLDEN_DIFFS_PPS, offset_wrt_start_trigger_flir/1e9)
print(f"Offset of first sample wrt start of trigger sequence: {offset_wrt_start_trigger_flir}")

print(f"Time offset ROS ts: {time_offset_ros_stamp}")

# Plot the time differencies
plt.figure()
plt.plot(np.diff(arr_camtime_double))
plt.show()

## Computation of offset for VNAV IMU

In [ ]:
# Create a bag reader only for VNAV msgs
bag_reader = BagReader(str(BAG), [const.VNAV_TOPIC_COMMON])
bag_reader.print_stats()

num_msgs = bag_reader.get_num_msgs(const.VNAV_TOPIC_COMMON)
# Cap to 60 seconds of data
if (num_msgs > 400*60):
    num_msgs = 400*60
arr_syncincnt = np.zeros(num_msgs, dtype=np.uint32)
arr_timesyncin = np.zeros(num_msgs, dtype=np.uint64) #ns
arr_timestartup = np.zeros(num_msgs, dtype=np.uint64)
arr_timegps = np.zeros(num_msgs, dtype=np.uint64)
arr_timegpspps = np.zeros(num_msgs, dtype=np.uint16)

for i, (topic, msg, t_rec)  in enumerate(bag_reader.read_all()):
    if (i == num_msgs):
        break

    arr_syncincnt[i] = msg.syncincnt
    arr_timesyncin[i] = msg.timesyncin
    arr_timestartup[i] = msg.timestartup
    arr_timegps[i] = msg.timegps
    arr_timegpspps[i] = msg.timegpspps
    if (i % 2500 == 0):
        print(f"{i} - {i/num_msgs}")

arr_timesyncin_double = arr_timesyncin.astype(np.double)/1e9 #us
arr_timestartup_double = arr_timestartup.astype(np.double)/1e9 #us
# General plot of the syncincounter
#plt.figure()
#plt.plot(arr_timesyncin/5e3)
#plt.plot(arr_syncincnt)
#plt.show()

In [ ]:
# The VN is running at 400 Hz, and the sync pulse happens every 50 Hz. This means that there are 
# 8 samples in which syncincnt does not increase.
# It also means that the maximum timesyncin difference is 1/50 if we have a constant set of pulses

# Find the samples that have more than 1/50 for arr_timesyncin
greater_than_trigger_period = np.where(arr_timesyncin_double > 1/const.TRIGGER_FREQ)[0]

# Find when we have a discontinuity indicating gaps. The +1 is because the we are detecting the last stable sample of a group
gaps = np.where(np.diff(greater_than_trigger_period) > 1)[0] + 1

# Also add the first one 
gaps_idx = np.concatenate((np.array([greater_than_trigger_period[0]]), greater_than_trigger_period[gaps]))

# gaps_idx correspond to 8 samples after the trigger happened. Correct this offset
gaps_idx -= 8

# Verify that these indices have timesyncin that is less than 1/8 of the period of the trigger
assert np.all(arr_timesyncin_double[gaps_idx] < (1/8*1/const.TRIGGER_FREQ))

# Get all the sensor times for the gaps
time_diffs_vnav = np.diff(arr_timestartup_double[gaps_idx] - arr_timesyncin_double[gaps_idx])
diff_with_golden_vnav = time_diffs_vnav - const.GOLDEN_DIFFS
print(f"Golden diffs: {const.GOLDEN_DIFFS}")
print(f"This bag diffs: {time_diffs_vnav}")
print(f"Diff with golden in us: {diff_with_golden_vnav*1e6}")

# Timing is good when is less than 10 us
assert(np.all(diff_with_golden_vnav*1e6<10))

# Get the first sample after 207.0027 ms ~= 82/83
if arr_timesyncin[np.max(gaps_idx[3])+82] < arr_timesyncin[np.max(gaps_idx[3])+83]:
    first_sample_vnav = np.max(gaps_idx[3]) + 82 
else:
    first_sample_vnav = np.max(gaps_idx[3]) + 83
    
# Confirm that the offset is correct
assert (arr_syncincnt[first_sample_vnav] - arr_syncincnt[gaps_idx[3]]) == 1
assert (arr_syncincnt[first_sample_vnav+1] - arr_syncincnt[first_sample_vnav]) == 0
assert (arr_syncincnt[first_sample_vnav-1] - arr_syncincnt[gaps_idx[3]]) == 0

# Get time offset
time_offset_vnav = arr_timestartup[first_sample_vnav] - arr_timesyncin[first_sample_vnav]
print(f"Time offset: {time_offset_vnav}")
offset_wrt_start_trigger_vnav = arr_timestartup[first_sample_vnav] - arr_timesyncin[first_sample_vnav] - (arr_timestartup[np.min(gaps_idx)] - arr_timesyncin[np.min(gaps_idx)])
assert np.isclose(const.GOLDEN_DIFFS_PPS, offset_wrt_start_trigger_vnav/1e9)
print(f"Offset of first sample wrt start of trigger sequence: {offset_wrt_start_trigger_vnav}")

# Timestamp of the first sample
print(f"Timestamp of first sample: {arr_timestartup[first_sample_vnav] - time_offset_vnav}") 
print(f"Timesyncin of first sample: {arr_timesyncin[first_sample_vnav]}")

In [ ]:
plt.figure()
fig, ax = plt.subplots(nrows=2, ncols=2)
for i, idx in enumerate(gaps_idx):
    ax[i//2, i%2].plot(np.arange(idx-1, idx+10), arr_timesyncin[idx-1:idx+10])
    ax[i//2, i%2].plot(idx, arr_timesyncin[idx], 'or')
plt.show()


## Computation of offset Event Camera

In [ ]:
# Create a bag reader only for EC msgs
bag_reader = BagReader(str(BAG), [const.EC_TOPIC])
bag_reader.print_stats()

decoder = ECDecoder()
# Use the flir trigger to get the event numbers. These should be roughly the same * 2
num_msgs = num_msgs_flir*2
triggers_ec = np.zeros(num_msgs, dtype=np.uint64)

i = 0
for topic, msg, t_rec  in bag_reader.read_all():
    decoder.decode(msg)
    # cd_events = decoder.get_cd_events()
    # print(cd_events)
    trig_events = decoder.get_ext_trig_events()
    if len(trig_events) > 0 and trig_events[0][0] == 1:
        assert len(trig_events) == 1
        triggers_ec[i] = trig_events[0][1]
        i += 1
        if (i == num_msgs):
            break

triggers_ec_double = triggers_ec.astype(np.double)[:i] / 1e6

In [ ]:
# Calculate the offsets for the event camera
diffs_ec = np.diff(triggers_ec_double)
p99 = np.percentile(diffs_ec, 99)
assert np.isclose(1/const.TRIGGER_FREQ, p99, atol=1e-6, rtol=1e-4)

# Calculate the indices of the peaks
peak_idx = np.where(diffs_ec > 3*1/const.TRIGGER_FREQ)[0]
print(peak_idx)

# Plot the time differencies
plt.figure()
plt.plot(np.diff(triggers_ec_double[:1000]))
plt.show()
assert len(peak_idx) == 4

# The peak corresponds to the last sample before and after the silence
time_diffs_ec = np.diff(triggers_ec_double[peak_idx])

diff_with_golden_ec = time_diffs_ec - const.GOLDEN_DIFFS
print(f"Golden diffs: {const.GOLDEN_DIFFS}")
print(f"This bag diffs: {time_diffs_ec}")
print(f"Diff with golden in us: {diff_with_golden_ec*1e6}")

# Timing is good when is less than 10 us
assert(np.all(diff_with_golden_ec*1e6<10))

# Time offset is the time of the first diff
first_sample_ec = peak_idx[3] + 1
offset_wrt_start_trigger_ec = triggers_ec[first_sample_ec] -triggers_ec[peak_idx[0]]
print(f"Offset of first sample wrt start of trigger sequence: {offset_wrt_start_trigger_ec}")
assert np.isclose(const.GOLDEN_DIFFS_PPS, offset_wrt_start_trigger_ec/1e6)
time_offset_ec_pre_cut = triggers_ec[first_sample_ec]

# As we will cut the event stream, we need to get a new timestamp for the offset (post cut)
# Read https://github.com/ros-event-camera/event_camera_codecs/?tab=readme-ov-file#event-time-stamps for more information
bag_reader = BagReader(str(BAG), [const.EC_TOPIC])
decoder = ECDecoder()
i = 0
for topic, msg, t_rec  in bag_reader.read_all():
    decoder.decode(msg)
    # cd_events = decoder.get_cd_events()
    # print(cd_events)
    trig_events = decoder.get_ext_trig_events()
    if len(trig_events) > 0 and trig_events[0][0] == 1:
        if i < first_sample_ec:
            i += 1
        else:
            assert trig_events[0][1] == time_offset_ec_pre_cut
            # Instantiate a new decoder and get the time offset
            decoder = ECDecoder()
            decoder.decode(msg)
            trig_events = decoder.get_ext_trig_events()
            time_offset_ec_post_cut = trig_events[0][1]
            break

print(f"Time offset pre cut: {time_offset_ec_pre_cut}")
print(f"Time offset post cut: {time_offset_ec_post_cut}")

# Print timestamp of the first sample to double check
# print(f"Old timestamp of first sample: {triggers_ec[first_sample_ec_old] - time_offset_ec}") 

# Computation of offset for SF000 range sensor

In [ ]:
# We will use the timestamp of the first camera message for the time offset
time_offset_sf000 = time_offset_ros_stamp[0]*1000000000 + time_offset_ros_stamp[1]

# Computation of offset for UBLOX

In [ ]:
# Create a bag reader only for UBLOX
bag_reader = BagReader(str(BAG), [const.TIM_TM2_TOPIC, const.NAVPVT_TOPIC])
bag_reader.print_stats()
tim_tm2_number = bag_reader.get_num_msgs(const.TIM_TM2_TOPIC)

triggers_ublox_ns = np.zeros(tim_tm2_number, dtype=np.int64)

i = 0
# navpvt_ublox_nano = np.zeros(bag_reader.get_num_msgs(const.NAVPVT_TOPIC), dtype=np.int64)
# navpvt_ublox_ms = np.zeros(bag_reader.get_num_msgs(const.NAVPVT_TOPIC), dtype=np.int64)
# j = 0
for topic, msg, t_rec  in bag_reader.read_all():
    if topic == const.TIM_TM2_TOPIC:
        # only count rising edge messages
        if msg.rising_edge_count == 0:
            continue
        # There is a 18 second offset between the TIM_TM2 and the NAVPVT message (GNSS vs UTC time)
        triggers_ublox_ns[i] = msg.tow_ms_r*1000000 + msg.tow_sub_ms_r + 18000000000
        i+=1
        if (i % 50 == 0):
            print(f"{i} - {i/tim_tm2_number}")
        if i == tim_tm2_number:
            break
    # elif topic == const.NAVPVT_TOPIC:
    #     navpvt_ublox_nano[j] =  msg.nano # msg.i_tow*1000000 +
    #     navpvt_ublox_ms[j] = msg.i_tow*1000000
    #     j+=1

# napvt_ublox_ns = navpvt_ublox_ns[:j]
triggers_ublox_ns = triggers_ublox_ns[:i]
triggers_ublox_double = triggers_ublox_ns/1e9

In [ ]:
# Find the period where the gap happens
up_idx = np.where(np.diff(triggers_ublox_double) > 1.5)[0][0]
first_sample_ublox = up_idx+1

# Compare this period with the golden diff
offset_wrt_start_trigger_ublox = triggers_ublox_ns[first_sample_ublox] - triggers_ublox_ns[up_idx]
assert np.isclose(const.GOLDEN_DIFFS_PPS, offset_wrt_start_trigger_ublox/1e9, atol=1e-6, rtol=1e-4)

# Compute time offset
time_offset_ublox = triggers_ublox_ns[first_sample_ublox]

# Compute drift. Skip first and last sample for better results
diff_double = triggers_ublox_double[-1] - triggers_ublox_double[up_idx+1]
drift_ns_array = np.diff(triggers_ublox_ns[up_idx+2:]) - 1000000000
# filter lost samples
drift_ns_array = drift_ns_array[drift_ns_array < 10000000] # 10 ms
assert np.abs(np.mean(drift_ns_array)) < 10000 # 10 ppm
print(f"Estimated drift ns - mean: {np.mean(drift_ns_array)} - min: {np.min(drift_ns_array)} - max: {np.max(drift_ns_array)} - std: {np.std(drift_ns_array)}")

# Analysis of results and write to yaml

In [ ]:
# Compare the timings for all the sensors
print("Timing comparison:")
print(f"EC: {diff_with_golden_ec*1e6}")
print(f"FLIR: {diff_with_golden_flir*1e6}")
print(f"VNAV: {diff_with_golden_vnav*1e6}")

In [ ]:
print("Gap timing comparison:")
print(f"EC: {offset_wrt_start_trigger_ec*1000}")
print(f"FLIR: {offset_wrt_start_trigger_flir}")
print(f"VNAV: {offset_wrt_start_trigger_vnav}")
print(f"UBLOX: {offset_wrt_start_trigger_ublox}")


In [ ]:
# Save into yaml
time_offsets = {"offsets": {
                "ec":    {"time_offset_pre_cut": int(time_offset_ec_pre_cut), "time_offset_post_cut": int(time_offset_ec_post_cut), "first_sample": int(first_sample_ec)},
                "flir":  {"time_offset": int(time_offset_flir), "first_sample": int(first_sample_flir)},
                "vnav":  {"time_offset": int(time_offset_vnav), "first_sample": int(first_sample_vnav)},
                "sf000": {"time_offset": int(time_offset_sf000)},
                "ublox": {"time_offset": int(time_offset_ublox), "first_sample": int(first_sample_ublox)}},
                "metadata": {
                    "date_modified": str(datetime.datetime.now()),
                    "source_bag": str(BAG.name)},
                "timing_info": {
                "ec":    {"pps_gap_ns": int(offset_wrt_start_trigger_ec*1000), "intervals_ns": [int(i) for i in diff_with_golden_ec*1e9]},
                "flir":  {"pps_gap_ns": int(offset_wrt_start_trigger_flir), "intervals_ns": [int(i) for i in diff_with_golden_flir*1e9]},
                "vnav":  {"pps_gap_ns": int(offset_wrt_start_trigger_vnav), "intervals_ns": [int(i) for i in diff_with_golden_vnav*1e9]},
                "ublox": {"pps_gap_ns": int(offset_wrt_start_trigger_ublox)}
                },
                "drift_per_s_wrt_gps_ns": {"mean": float(np.mean(drift_ns_array)),
                                           "std": float(np.std(drift_ns_array)),
                                           "min": int(np.min(drift_ns_array)),
                                           "max": int(np.max(drift_ns_array))}
               }
            
with open(SYNC_YAML, "w") as file:
    yaml.dump(time_offsets, file)